In [2]:
import json
from pathlib import Path

import pandas as pd

### Exp 1 and 3: no 4D SHs and no 4D rot

In [8]:
#results_path = Path('output/WAT/realtime4DGS-exp1/force_sh_3d_true')
#results_path = Path('output/WAT/realtime4DGS-exp1/force_sh_3d_false')
#results_path = Path('output/WAT/realtime4DGS-exp3/gaussians_2000000/force_sh_3d_true')
#results_path = Path('output/WAT/realtime4DGS-exp3/gaussians_2000000/rot_4d_false')
#results_path = Path('output/WAT/realtime4DGS-exp3/gaussians_1000000/force_sh_3d_true')
results_path = Path('output/WAT/realtime4DGS-exp3/gaussians_1000000/rot_4d_false')

In [9]:
results = []
for scene in sorted(results_path.iterdir()):
    # Storage
    checkpoint = scene / 'chkpnt30000.pth'
    if not checkpoint.exists():
        print(f"{checkpoint.name} for scene {scene.name} doesn't exist")
        continue
    storage_gb= round(checkpoint.stat().st_size / 1024**3, 3)

    # Wanbd metrics
    wandb_run_path = scene / 'wandb' / 'latest-run' / 'files'
    metadata_json_path = wandb_run_path / 'wandb-metadata.json'
    summary_json_path = wandb_run_path / 'wandb-summary.json'

    if not metadata_json_path.exists() or not summary_json_path.exists():
        print(f"Wandb json paths can't be found for {scene.name}")

    with open(summary_json_path, 'r') as f:
        data = json.load(f)
        
    metrics = {
        'Scene': scene.name,
        'PSNR': data.get('test_psnr', 'N/A'),
        'SSIM': data.get('test_ssim', 'N/A'),
        'Runtime (min)': int(data.get('_runtime') / 60),
        'Total Gaussians': data.get('total_points', 'N/A'),
        'Storage (GB)': storage_gb,
        }
    
    results.append(metrics)

In [10]:
results

[{'Scene': 'breville',
  'PSNR': 31.209435070262234,
  'SSIM': 0.9340623354210573,
  'Runtime (min)': 161,
  'Total Gaussians': 2012325,
  'Storage (GB)': 1.492},
 {'Scene': 'car_resized',
  'PSNR': 23.077088869535007,
  'SSIM': 0.6527707519439551,
  'Runtime (min)': 131,
  'Total Gaussians': 2074624,
  'Storage (GB)': 1.538},
 {'Scene': 'community',
  'PSNR': 25.23020252938998,
  'SSIM': 0.7736080400014328,
  'Runtime (min)': 187,
  'Total Gaussians': 2095591,
  'Storage (GB)': 1.554},
 {'Scene': 'dyson',
  'PSNR': 25.665050324939546,
  'SSIM': 0.9161116225378854,
  'Runtime (min)': 130,
  'Total Gaussians': 2020853,
  'Storage (GB)': 1.498},
 {'Scene': 'grill_resized',
  'PSNR': 25.530934638165412,
  'SSIM': 0.7487825205985535,
  'Runtime (min)': 125,
  'Total Gaussians': 2035141,
  'Storage (GB)': 1.509},
 {'Scene': 'kitchen',
  'PSNR': 28.830511229378835,
  'SSIM': 0.9246994171823774,
  'Runtime (min)': 162,
  'Total Gaussians': 2050919,
  'Storage (GB)': 1.52},
 {'Scene': 'living_

In [11]:
df = pd.DataFrame(results)
df

,Scene,PSNR,SSIM,Runtime (min),Total Gaussians,Storage (GB)
0,breville,31.209435,0.934062,161,2012325,1.492
1,car_resized,23.077089,0.652771,131,2074624,1.538
2,community,25.230203,0.773608,187,2095591,1.554
3,dyson,25.665050,0.916112,130,2020853,1.498
4,grill_resized,25.530935,0.748783,125,2035141,1.509
5,kitchen,28.830511,0.924699,162,2050919,1.520
6,living_room,26.096525,0.891480,143,2055374,1.524
7,mac,30.731632,0.942843,126,2010690,1.491
8,ninja,29.041838,0.931424,124,2006455,1.487
9,spa,29.079381,0.904781,167,2023954,1.500


In [12]:
csv_filename = '_'.join(str(results_path).split('/')[1:])
df.to_csv(f'{csv_filename}.csv', index=False)
print(f'file written to {csv_filename}')

file written to WAT_realtime4DGS-exp3_gaussians_2000000_force_sh_3d_true


### Exp 2: number of gaussians

In [2]:
results_base_path = Path('output/WAT/realtime4DGS-exp2')

In [ ]:
gaussian_counts_paths = sorted([path for path in results_base_path.iterdir() if path.is_dir()], key=lambda x: int(x.name.split('_')[-1]))
gaussian_counts_paths

In [10]:
# Helper functions
def get_nvs_metrics(scene_path: Path) -> dict:
    json_path = scene_path / 'results.json'
    if not json_path.exists():
        print(f"{json_path.name} for scene {json_path.parent.name} doesn't exist")
    with open(json_path, 'r') as f:
        data = json.load(f)
    data = data.get('ours_None', {})
    result = {
        'Gaussians': int(scene_path.parent.name.split('_')[-1]),
        'Scene': scene_path.name,
        'PSNR': data.get('PSNR', 'N/A'),
        'SSIM': data.get('SSIM', 'N/A'),
        'LPIPS (vgg)': data.get('LPIPS', 'N/A')
    }
    return result

def get_run_metrics(scene_path: Path) -> dict:
    wandb_run_path = scene_path / 'wandb' / 'latest-run' / 'files'
    metadata_json_path = wandb_run_path / 'wandb-metadata.json'
    summary_json_path = wandb_run_path / 'wandb-summary.json'

    if not metadata_json_path.exists() or not summary_json_path.exists():
        print(f"Wandb json paths can't be found for {scene_path.name}")
    
    with open(metadata_json_path, 'r') as f:
        data = json.load(f)
        gpu_model = data.get('gpu_nvidia')[0]['name']

    with open(summary_json_path, 'r') as f:
        data = json.load(f)
        try:
            runtime = int(data.get('_runtime') / 60)
        except:
            runtime = 'N/A'

    result = {
        'runtime (min)': runtime,
        'GPU model': gpu_model,
     }
    return result

def get_storage(scene_path: Path) -> float:
    checkpoint = scene_path / 'chkpnt30000.pth'
    if not checkpoint.exists():
        print(f"{checkpoint.name} for scene {scene_path.name} doesn't exist")
    storage_mb = round(checkpoint.stat().st_size / 1024**3, 3)
    return storage_mb

In [11]:
metrics = []
for path in gaussian_counts_paths:
    for scene in sorted(path.iterdir()):
        scene_metrics = get_nvs_metrics(scene)
        scene_metrics.update(get_run_metrics(scene))
        scene_metrics.update({'Storage (GB)': get_storage(scene)})
        metrics.append(scene_metrics)

In [ ]:
df = pd.DataFrame(metrics)
df

In [14]:
df.to_csv('WAT_realtime4DGS-exp2.csv', index=False)

In [ ]:
df.groupby('Gaussians')[['runtime (min)', 'Storage (GB)']].mean()